Evaluate a given model and compare multiple models. Push selected models to Neptune.

In [ ]:
import importlib
import neptune
import pandas as pd
import numpy as np
import datetime as dt
import torch

from src import utils
from src import data
from src import backtest
from src import constants
from matplotlib.ticker import MultipleLocator
from matplotlib.dates import DayLocator, AutoDateLocator, ConciseDateFormatter
%matplotlib inline

ARCHS_DIR = 'archs'
DATA_DIR = 'data'
EXPERIMENTS_DIR = 'experiments'
DEVICE='cpu'
NEPTUNE_PRJ = 'indiacovidseva/covid-net'

### Backtest one model

In [ ]:
experiment_id = '0001'
checkpoint = 'latest-e1740.pt'

model, cp = utils.load_model(experiment_id, checkpoint)

#### Plot loss and acc

In [ ]:
df_loss = pd.DataFrame({
    'trn_loss': cp['trn_losses'],
    'val_loss': cp['val_losses']
})

df_acc = pd.DataFrame({
    'trn_acc': cp.get('trn_acc', np.zeros((cp['config']['NUM_EPOCHS']))),
    'val_acc': cp.get('val_acc', np.zeros((cp['config']['NUM_EPOCHS'])))
})

# smoothen
df_loss['trn_loss'] = df_loss['trn_loss'].rolling(3, min_periods=1, center=True).mean()
df_loss['val_loss'] = df_loss['val_loss'].rolling(3, min_periods=1, center=True).mean()
df_acc['trn_acc'] = df_acc['trn_acc'].rolling(3, min_periods=1, center=True).mean()
df_acc['val_acc'] = df_acc['val_acc'].rolling(3, min_periods=1, center=True).mean()

_ = df_loss.plot(
    y=['trn_loss', 'val_loss'],
    title='Loss per epoch',
    subplots=False,
    figsize=(5,5),
    sharex=False,
    logy=True
)
_ = df_acc.plot(
    y=['trn_acc', 'val_acc'],
    title='Acc per epoch',
    subplots=False,
    figsize=(5,5),
    sharex=False,
    logy=True
)

#### Load training data and backtest

In [ ]:
cols = ['location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'population', 'population_density', 'gdp_per_capita', 'hospital_beds_per_thousand', 'median_age']
dates = ['date']
df = pd.read_csv(DATA_DIR + "/" + cp['config']['DS']['SRC'],
                 usecols=cols,
                 parse_dates=dates)
df = data.fix_anomalies_owid(df)
df.sample()

In [ ]:
backtest.countrywise(model, cp, df, constants.STT_INFO)

#### Load covid19india data and backtest

In [ ]:
df = data.get_statewise_data()

In [ ]:
backtest.statewise(model, cp, df, constants.STT_INFO, plot=False)

#### Upload model to Neptune

In [ ]:
neptune_prj = neptune.init(NEPTUNE_PRJ)
neptune_exp = neptune_prj.get_experiments(id=cp['config']['NEPTUNE_ID'])[0]
neptune_exp.log_artifact(EXPERIMENTS_DIR + "/" + experiment_id + "/" + checkpoint)

### Backtest all models

In [ ]:
accs = []
for e in range(0, 10001, 1): # start, stop, step
    checkpoint = 'latest-e' + str(e) + '.pt'
    try:
        model, cp = utils.load_model(experiment_id, checkpoint, v=False)
        acc = backtest.countrywise(model, cp, df, plot=False)
        accs.append(acc)
        print(checkpoint, acc)
    except Exception as e:
        print(checkpoint, e)

#### Plot test accuracy vs epochs

In [ ]:
# which output feature's acc should be plotted
feature = 0
feature_name = cp['config']['DS']['FEATURES'][feature]

df_exp = pd.DataFrame({
    'test_acc': np.array(accs)[:, feature],
    'val_acc': cp.get('val_acc', np.zeros((cp['config']['NUM_EPOCHS']))),
    'epochs': np.arange(0, 10001, 1)
})
ax = df_exp[:].plot(
    x='epochs',
    y=['test_acc', 'val_acc'],
    title='Test accuracy: ' + feature_name,
    subplots=False,
    figsize=(5,5),
    sharex=False
)

print("Models with best test accuracy for", feature_name)
print(df_exp.sort_values('test_acc', ascending=False).head())
# print(df_exp.loc[(df_exp['test_acc'] - df_exp['val_acc']).abs().sort_values(ascending=True).index]\
#       .head(100).sort_values('test_acc', ascending=False))

#### Log test accuracy to Neptune

In [ ]:
neptune_prj = neptune.init(NEPTUNE_PRJ)
neptune_exp = neptune_prj.get_experiments(id=cp['config']['NEPTUNE_ID'])[0]
for idx, row in df_exp.iterrows():
    neptune_exp.log_metric('test accuracy: ' + feature_name, row['epochs'], row['test_acc'])